# Week 6: Neural Oscillations

Note: you're going to need to install ***mne*** for this workshop. Use pip to do this!

In [ ]:
%load_ext autoreload
%autoreload 2

import csv
import mne
import matplotlib.pyplot as plt
import numpy as np

import biosppy
from scipy import signal

Congrats! So far, you've filtered your first EEG signal, explored a bit of neuroscience, learned about how to choose your filters, and learned about power spectral analysis! Now, we're going to try and use power spectral analysis to interpret some EEG.

We'll be giving you two samples of EEG. Your goal will be to decipher what's going on in each of these samples. *Let the investigation begin!*

EEG_SAMPLE_1 is available in this repository. You can download EEG_SAMPLE_2 from this link: http://bit.ly/ntuoft-eeg-sample-2

Often times, EEG is saved as an ***FIF*** file, which is a (much more) standard file format than CSV for EEG data. We're going to use a library called **mne** to read in the FIF file (MNE is actually a really epic library for EEG and MEG analysis, but it's a bit too much for the scope of this course). First, let's load in the data:

In [ ]:
EEG_SAMPLE_1 = "./../data/eeg_sample_1.fif"
EEG_SAMPLE_2 = "./../data/eeg_sample_2.fif"

In [ ]:
eeg_fif_1 = mne.io.read_raw_fif(EEG_SAMPLE_1)
eeg_fif_2 = mne.io.read_raw_fif(EEG_SAMPLE_2)

Next, we're going to need a plotting function again, so let's write one up! You should know how to do this by now, so we made it for you :)

In [ ]:
def plot_eeg(t, eeg):
    plt.rcParams['figure.figsize'] = [140, 12]

    fig = plt.figure()
    
    ax_start = 0.1
    ax_step = (0.9 - 0.1) / eeg.shape[0] # Divide graph into channels
    
    axes= []
    colours = ['#A283C4', '#8B2BC4', '#3978E0', '#FFA500', '#3CB2BA', '#FF7685']
    
    for i in range(eeg.shape[0]):
        axes.append(fig.add_axes([0.1, ax_start + i * ax_step, 0.9, ax_step]))
    
    for i in range(eeg.shape[0]):
        axes[i].plot(t, eeg[eeg.shape[0] - i - 1], color=colours[i % len(colours)])

    plt.xticks(np.arange(t[0], t[-1], 1.0))

    fig.show()

MNE loads data into things called **Raw objects**. They actually operate just like lists for the most part, but also have some other nifty features like saving the sampling frequency of the data, labels for different events (like eye blink, bad data, etc). Here, we're going to extract data from 3.0 seconds in the sample all the way till 40.0 seconds. You're going end up with a list of lists:

`[left_ear_list, left_forehead_list, right_forehead_list, right_ear_list, empty_list, aux_electrode_list]`

You're only going to need the first 4 lists for this experiment, but the aux_electrode_list does contain some interesting stuff!

In [ ]:
time_start = 3.0
time_end = 40.0

sampling_freq_1 = eeg_fif_1.info['sfreq']

# Returns both EEG data and timestamps
eeg1, t1 = eeg_fif_1[:, int(sampling_freq_1 * time_start):int(sampling_freq_1 * time_end)]

# Plotting just the EEG channels: try this yourself!

Enough setup, let's get into the fun stuff! Last lecture, we mentioned that you can tell a lot about a person's head from looking at the *frequencies* of brainwaves and their associated *amplitudes* in **volts**. It turns out that a slightly more useful / standard comparison is *power vs frequencies*, where power is the amount of energy transferred per second. Both comparisons are analogous to each other but *power vs frequencies* is slightly more useful / makes a bit more sense. This comparison is called the ***power spectrum***, and the analysis you're about to do is called ***power spectral analysis***.

(Side question: why is it that the power spectrum and the FFT tells us similar thing in this case? Hint: Use Ohm's Law: V = IR)

Let's try it out! The following function will help you get the power spectrum of the EEG data:

```
# Note: this function returns both the power spectrum, and the frequencies the spectrum is associated with
freq_1_l_ear, psd_1_l_ear = biosppy.tools.power_spectrum(signal=eeg1[0], # Only accepts 1 channel of data
                   sampling_rate=sampling_freq_1,
                   pad=None,
                   pow2=False,
                   decibel=True) # 10 decibel increase -> increase in power by a FACTOR of 10
```
Try plotting power vs frequency! What do you notice?

In [ ]:
# Get your power spectrum here!


In [ ]:
plt.rcParams['figure.figsize'] = [14, 6]
# ... and graph your power spectrum here!


There's a massive spike at 60Hz, and 120 Hz! Where do you think that is coming from? Try cleaning the data with the methods we learned so far! Feel free to experiment with variables like the order of the filter, etc and see how that affects your data and power spectrum by plotting both.

Note: biosppy's filter function can accept multi-channel data! However, it expects the matrix to have dimensions of time * channels, instead of channels * time! You can 'rotate' the matrix like this: `eeg1.T`

Note 2: usually we'd want to do something called a **notch filter**, which removes a specific frequency. However, biosppy unfortunately doesn't have it. You can use MNE's notch filter if you like, but *you don't have to if you're already low-pass filtering the data*

`
mne.filter.notch_filter(eeg_list, sampling_freq, freq_to_notch) # eeg_list takes in all four channels! Dims channels * time
`

In [ ]:
def filter_data(data):
    # Filter your data here!
    
    return filtered_data

In [ ]:
filt_eeg1 = (filter_data(eeg1.T)).T
# filt_eeg = mne.filter.notch_filter(eeg, sampling_freq, 60.0) # Optionally, you can do this INSTEAD

In [ ]:
# Plot your data!

In [ ]:
# Try computing the power spectrum again! What does it look like? Adjust your filter_data as necessary


The data looks a lot nicer, and we can kind of tell what's happening in the brain! Question: what do you think the person is doing when this data is being collected? (This is a trick question :P ... you're going to have to look both at the time data and the power spectrum to solve this one).

Enter your answer below

In [ ]:
# What was the person doing? Enter your answer as a comment!
# Answer: _

Let's start summarizing information now, so that we can actually use it! We can ***average*** the power spectrum in ***frequency bands*** we are interested in! For example, we could get the average band powers in the theta, alpha, beta, etc power bands! There's a function in biosppy *specifically* to help you with this! Try finding it :), and plot what you find! What do you think is happening in this subject's brain, and what do you think they are doing?

In [ ]:
# Get your power bands here!

In [ ]:
# ... and plot them here! Plot 1 power band at a time (or you can use sub-plots to plot all of them if you want!)
# What are the units for the x-axis? y-axis?

Yay! We analyzed the first sample of EEG. Now let's try analyzing the second EEG dataset! (This dataset has 79 EEG channels! We're only going to explore one of them :)).

This time, don't use the biosppy.eeg.power_bands() function; use biosppy.tools.band_power instead! Here's the docstring:

```
def band_power(freqs=None, power=None, frequency=None, decibel=True):

    """Compute the avearge power in a frequency band.
    Parameters
    ----------
    freqs : array
        Array of frequencies (Hz) at which the power was computed.
    power : array
        Input power spectrum.
    frequency : list, array
        Pair of frequencies defining the band.
    decibel : bool, optional
        If True, input power is in decibels.

    Returns
    -------
    avg_power : float
        The average power in the band.
    """
```

Note: if you want more info about the dataset, you can try running the following: `eeg_fif.info` (for a Raw obj called 'eeg-fif' )

In [ ]:
sampling_freq_2 = eeg_fif_2.info['sfreq']

# Returns both EEG data and timestamps. We're only going to analyze Fp1, Fpz, and Fp2 to make things easier :D
chs = [eeg_fif_2.ch_names.index('Fpz')]
eeg2, t2 = eeg_fif_2[chs, int(sampling_freq_2 * time_start):int(sampling_freq_2 * time_end)]

# Plotting just the EEG channels
plot_eeg(t2, eeg2)

In [ ]:
# Plot your power spectrum! What do you think the person is doing by looking at this data?
# Your code:

# Your answer: ___

So we said that we're going to use *biosppy.tools.band_power()* instead! By doing this we're going to learn some important concepts when it comes to analyzing live data.

First of all, it's kind of boring just to see the power spectrum overall! It would be really cool to explore how the power spectrum *changes* over time. However, from Nyquist's theorem we know that we need a certain number of data samples to analyze certain frequencies!

The answer lies in *windowing*. At time **t**, we can *look at the last n seconds of data* instead of the full dataset! If we slide our window through time (ie incrementing t, and looking at the last n seconds of data), we can see how the bandpowers are changing!

But what if something interesting happens between two windows! To fix that problem, we can have the windows overlap :)

Try it yourself! If you're stuck, don't worry! We're here to help you :)

In [ ]:
# What should your window length be? Use Nyquist's theorem to help you with this
# There should generally be an overlap of half your window size
window_length = 
overlap = 

band_powers = [] # Should have dims: n time points * 5 bands

# Each band has a low-freq and a high-freq, so you can represent bands as [low-freq, high-freq]
bands = [[_, _], [_, _], [_, _], [_, _], [_, _]] # [Delta, theta, alpha, beta, gamma]

# You data is in terms of indices, not times! At what indices do you want to take a window?
# Get the appropriate times that correlate to the indices as well; we'll need them for graphing
inds = _
times = _

# For each window:
    powers = [] # Band powers for this window
    # For each band we want to compute:
        # Compute your band powers!
    band_powers.append(powers)

# Plot your data! What do you notice?


Congrats on doing your first 'live' EEG analysis! Next session, we're going to try and make a BCI!